In [ ]:
%matplotlib widget

import ipywidgets as widgets
from ipywidgets import HBox, VBox, jslink, Box, Layout
from IPython.display import display, Latex, Image

import params as st
from model import linSys
from feedforward import *

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [ ]:
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 5px 5px 0px',
        padding='2px 2px 2px 2px'
     )

# Steuerung beim Zweimassenschwinger

In [ ]:
imag = Image("../images/zweimassenschwinger.png", width=400)
display(imag)

**Modellgleichungen**

\begin{align*}
    m (\ddot{y}_1(t) + \ddot{y}_2(t)) & = u(t) \\
    M \ddot{y}_2(t) + d \dot{y}_2(t) + k y_2(t) & = -u(t)
\end{align*}

**Definition Parameter**

In [ ]:
tSim = np.linspace(0, 2, 2001)

In [ ]:
output = widgets.Output()

with output:
    fig = plt.figure(figsize=(10, 5))
    ax3 = plt.subplot(222)
    ax4 = plt.subplot(224)
    ax1 = plt.subplot(221)
    ax2 = plt.subplot(223)

plt.subplots_adjust(wspace=0.2, hspace=0.3)
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.subplots_adjust(bottom=0.1, top=0.93, left=0.125, right=0.9)

ax1.set_xlim([tSim[0], tSim[-1]]) 
ax2.set_xlim([tSim[0], tSim[-1]]) 
ax4.set_xlim([tSim[0], tSim[-1]]) 
ax1.set_ylim([-15, 15]) 
ax2.set_ylim([-0.1, 1]) 
ax4.set_ylim([-10, 10]) 
ax1.grid() 
ax2.grid()
ax4.grid()
ax3.set_xlim(-0.3, 1.3)
ax3.set_ylim(-0.1, st.h * 2 + 0.1)
ax3.set_xticks([])
ax3.set_yticks([])
ax1.set_ylabel(r"$u$")
ax2.set_xlabel(r"$t$")
ax2.set_ylabel(r"$y_1$ in m")
ax4.set_ylabel(r"$y_2$ in mm")
ax4.set_xlabel(r"$t$")

lineY1, = ax2.plot([], [], label=r"ist")
lineY1ref, = ax2.plot([], [], '--', label=r"soll")
lineY2, = ax4.plot([], [])
lineY2ref, = ax4.plot([], [], '--')
lineU, = ax1.plot([], [])

pathAni = ax3.plot([-0.3, 1.3], [0, 0], zorder=0, color='C6')
lastAni = ax3.add_patch(plt.Rectangle((0, 0), st.b1, st.h, facecolor='0.', edgecolor='0.'))
wagenAni = ax3.add_patch(plt.Rectangle((0, st.h + 0.005), st.b2, st.h, facecolor='0.', edgecolor='0.'))

handlesAx, labelsAx = ax2.get_legend_handles_labels()
fig.legend([handle for i, handle in enumerate(handlesAx)],
           [label for i, label in enumerate(labelsAx)],
           bbox_to_anchor=(0.125, 0.94, 0.7735, .15), loc=3,
           ncol=2, mode="expand", borderaxespad=0., framealpha=0.5)

playB = widgets.Play(value=0,
                     min=0, 
                     max=len(tSim),
                     step=10)
sliderB = widgets.IntSlider(value=0,
                            min=0,
                            max=len(tSim),
                            step=10)
sliderT0 = widgets.FloatSlider(value=0,
                               min=0,
                               max=1,
                               step=0.1,
                               description=r'$t_0$')
sliderT = widgets.FloatSlider(value=0.75,
                               min=0.5,
                               max=1.5,
                               step=0.05,
                               description=r'$T$')
sliderYd = widgets.widgets.FloatSlider(value=0.5,
                                       min=0.2,
                                       max=1.0,
                                       step=0.1,
                                       description=r"$y_\text{d}$")
radioU = widgets.RadioButtons(options=['Trapez', 'S-Curve', 'Flach mit Filter'],
                              description='Eingang:',
                              disabled=False)

def updateOde(_):
    global res

    t0 = sliderT0.value
    T = sliderT.value
    yd = sliderYd.value

    def sys(t, x, u):
        params = st.d, st.k, st.m , st.M
        dx = linSys(t, x, u, params)

        return dx

    x0 = [0, 0, 0, 0]
    if radioU.value == 'Trapez':
        uIn = feedForwardTrapez(t0, T, yd)
    elif radioU.value == 'S-Curve':
        uIn = feedForwardTrapez(t0, T, yd)
    else:
        uIn = feedForwardTrapez(t0, T, yd)

    res = solve_ivp(sys,
                    [tSim[0], tSim[-1]],
                    x0,
                    t_eval=tSim,
                    args=(uIn, ))

def updatePlot(change):
    idx = change['new']

    y2 = res.y.T[idx, 2]
    y1 = res.y.T[idx, 0] - y2
    
    t0 = sliderT0.value
    T = sliderT.value
    yd = sliderYd.value
    if radioU.value == 'Trapez':
        uIn = feedForwardTrapez(t0, T, yd)
        lineU.set_data(tSim[:idx], [uIn(_t) for _t in tSim[:idx]])
        lineY1.set_data(tSim[:idx], (res.y.T[:idx, 0] - res.y.T[:idx, 2]))
        lineY1ref.set_data([], [])
        lineY2.set_data(tSim[:idx], res.y.T[:idx, 2] * 1000)
        lineY2ref.set_data([], [])
    elif radioU.value == 'S-Curve':
        uIn = feedForwardTrapez(t0, T, yd)
        lineU.set_data(tSim[:idx], [uIn(_t) for _t in tSim[:idx]])
        lineY1.set_data(tSim[:idx], (res.y.T[:idx, 0] - res.y.T[:idx, 2]))
        lineY1ref.set_data([], [])
        lineY2.set_data(tSim[:idx], res.y.T[:idx, 2] * 1000)
        lineY2ref.set_data([], [])
    else:
        uIn = feedForwardTrapez(t0, T, yd)
        lineU.set_data(tSim[:idx], [uIn(_t) for _t in tSim[:idx]])
        lineY1.set_data(tSim[:idx], (res.y.T[:idx, 0] - res.y.T[:idx, 2]))
        lineY1ref.set_data([], [])
        lineY2.set_data(tSim[:idx], res.y.T[:idx, 2] * 1000)
        lineY2ref.set_data([], [])

    lastAni.set_x(y2)
    wagenAni.set_x(y1)
    
    fig.canvas.draw()    

sliderB.observe(updatePlot, names='value')
sliderT0.observe(updateOde, names='value')
sliderT.observe(updateOde, names='value')
sliderYd.observe(updateOde, names='value')
radioU.observe(updateOde, names='value')

updateOde(_)

ffControls = VBox([sliderYd, sliderT0, sliderT])
ffControls.layout = make_box_layout()
inputControls = Box([radioU])
inputControls.layout = make_box_layout()
controls = HBox([ffControls, inputControls])

jslink((playB, 'value'), (sliderB, 'value'))
videoControls = VBox([HBox([playB, sliderB]), output])
videoControls.layout = make_box_layout()

HBox([controls, Box([videoControls])], layout=Layout(display='flex', flex_flow='row', justify_content='center', align_items='center'))